In [ ]:
from transformers import AutoTokenizer, AutoModelForCausalLM, Trainer, TrainingArguments, EarlyStoppingCallback, DataCollatorForLanguageModeling
import datasets
import pickle
import json
import random
import numpy as np

In [ ]:
import torch

In [ ]:
EXP = '1'

In [ ]:
import os

newpath = 'experiments/EXP' + EXP

if not os.path.exists(newpath):
    os.makedirs(newpath)

if not os.path.exists(newpath + 'model'):
    os.makedirs(newpath + 'model')

### Preprocessing

In [ ]:
import networkx as nx

graph = nx.read_graphml("../graph.graphml")

In [ ]:
len(graph)

In [ ]:
communities = nx.community.louvain_communities(graph, seed=42)

In [ ]:
lengths = []

for c in communities:
    lengths.append(len(c))

lengths = sorted(lengths, reverse=True)

In [ ]:
s = set()
for e in graph.edges(data=True):
    s.add(e[2]['display_relation'])

In [ ]:
s

In [ ]:
s = set()
for n in graph.nodes(data=True):
    s.add(n[1]['node_type'])

In [ ]:
s

### Train set

In [ ]:
train_set = []
m_n_values = []

for comm in communities:
    sub = graph.subgraph(comm)
    if not nx.is_connected(sub):
        c_comp = list(nx.connected_components(sub))
        for c in c_comp:
            communities.append(c)
        continue

    m = int(nx.diameter(sub))

    if m > 0:
        n = round(sub.number_of_nodes())

        if n == 0:
            n = 1
            
        random_path = nx.generate_random_paths(sub, n, path_length=m, seed=42)
        paths = []
        for rp in random_path:
            paths.append(rp)
        l = len(paths)
        if l > 0:
            train_set.append(paths)
            m_n = (m, n)
            m_n_values.append(m_n)

In [ ]:
m_n_values[0]

In [ ]:
with open('experiments/EXP' + EXP + '/m_n_values.pkl', 'wb') as f:
    pickle.dump(m_n_values, f)

In [ ]:
len(train_set)

In [ ]:
train_set[0][0]

In [ ]:
train_dataset = []
triples = set()

for walks in train_set:
    for walk in walks:
        chat = []

        system = {"role": "system", "content": "You are a chatbot that has to predict the relationship between nodes."}
        chat.append(system)
        for i in range(len(walk)-1):
            rel = graph[walk[i]][walk[i+1]]['display_relation']
            a = graph.nodes[walk[i]]['node_name']
            b = graph.nodes[walk[i+1]]['node_name']
            triple = (a, rel, b)
            triples.add(triple)
            
            user = "Which is the relationship between the node '" + a + "' and the node '" + b + "'?"
            assistant = rel
            
            message = {}
            message["role"] = "user"
            message["content"] = user
            chat.append(message)

            message = {}
            message["role"] = "assistant"
            message["content"] = assistant
            chat.append(message)

        if chat not in train_dataset:
            train_dataset.append(chat)

In [ ]:
len(triples)

In [ ]:
with open('experiments/EXP' + EXP + '/train_val_triples.pkl', 'wb') as file1:
    pickle.dump(triples, file1)

In [ ]:
len(train_dataset)

In [ ]:
import random

random.shuffle(train_dataset)

In [ ]:
l = int(len(train_dataset)/10)
val_dataset = train_dataset[:l]
train_dataset = train_dataset[l:]

In [ ]:
len(train_dataset)

In [ ]:
with open('experiments/EXP' + EXP + '/train_dataset.pkl', 'wb') as file1:
    pickle.dump(train_dataset, file1)

In [ ]:
with open('experiments/EXP' + EXP + '/val_dataset.pkl', 'wb') as file2:
    pickle.dump(val_dataset, file2)

### Test set

In [ ]:
test_triples = set()
edges = list(graph.edges(data=True))

while len(test_triples) < 1500:
    e = random.randint(0, 4049642)
    edge = edges[e]
    node1 = graph.nodes[edge[0]]['node_name']
    node2 = graph.nodes[edge[1]]['node_name']
    triple = (node1, edge[2]['display_relation'], node2)
    if triple not in triples:
        test_triples.add(triple)

In [ ]:
len(test_triples)

In [ ]:
abc = triples & test_triples
len(abc) # Numero di triple in comune tra train e test set

In [ ]:
with open('experiments/EXP' + EXP + '/test_triples.pkl', 'wb') as file1:
    pickle.dump(test_triples, file1)

In [ ]:
test_dataset = []
ground_truth = []

for t in test_triples:
    chat = []
    system = {"role": "system", "content": "You are a chatbot that has to predict the relationship between nodes."}
    user = {"role": "user", "content": "Which is the relationship between the node '" + t[0] + "' and the node '" + t[2] + "'?"}
    chat.append(system)
    chat.append(user)
    test_dataset.append(chat)

    assistant = {"role": "assistant", "content": t[1]}
    chat.append(assistant)
    ground_truth.append(chat)

In [ ]:
with open('experiments/EXP' + EXP + '/test_dataset.pkl', 'wb') as file1:
    pickle.dump(test_dataset, file1)

In [ ]:
with open('experiments/EXP' + EXP + '/ground_truth.pkl', 'wb') as file1:
    pickle.dump(ground_truth, file1)

### Acquisizione modello e dati

In [ ]:
with open('experiments/EXP' + EXP + '/train_dataset.pkl', 'rb') as file:
    train_dataset = pickle.load(file)

In [ ]:
with open('experiments/EXP' + EXP + '/val_dataset.pkl', 'rb') as file:
    val_dataset = pickle.load(file)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained("sapienzanlp/Minerva-350m-base-v1.0")
model = AutoModelForCausalLM.from_pretrained("sapienzanlp/Minerva-350m-base-v1.0")
tokenizer.pad_token = tokenizer.eos_token

In [ ]:
from datasets import Dataset

dataset1 = Dataset.from_dict({"chat": train_dataset})
dataset1 = dataset1.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=True, add_generation_prompt=False)})

dataset2 = Dataset.from_dict({"chat": val_dataset})
dataset2 = dataset2.map(lambda x: {"formatted_chat": tokenizer.apply_chat_template(x["chat"], tokenize=True, add_generation_prompt=False)})

### Training

In [ ]:
training_args = TrainingArguments(
    output_dir='outputs',
    num_train_epochs=10,
    per_device_train_batch_size=2,
    save_steps=250,
    save_total_limit=3,
    load_best_model_at_end=True,
    greater_is_better=False,
    do_eval=True,
    metric_for_best_model="eval_loss",
    eval_strategy="steps",
    logging_steps=250,
    warmup_steps=50,
    prediction_loss_only=True,
    fp16=True
)

trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=dataset1['formatted_chat'],
    eval_dataset=dataset2['formatted_chat'],
    data_collator=DataCollatorForLanguageModeling(tokenizer, mlm=False),
    callbacks=[EarlyStoppingCallback(early_stopping_patience=3)]
)

trainer.train()

In [ ]:
model.save_pretrained('experiments/EXP' + EXP + 'model')
tokenizer.save_pretrained('experiments/EXP' + EXP + 'model')

### Metrics

In [ ]:
def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed_all(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = False

set_seed(42)

In [ ]:
tokenizer = AutoTokenizer.from_pretrained('experiments/EXP' + EXP + 'model')
final_model = AutoModelForCausalLM.from_pretrained('experiments/EXP' + EXP + 'model')

In [ ]:
with open('experiments/EXP' + EXP + '/test_dataset.pkl', 'rb') as file:
    test_dataset = pickle.load(file)

In [ ]:
post_ft = []

for i in range(len(test_dataset)):
    prompt = []
    ground_truth = ''
    for m in test_dataset[i]:
        if m['role'] != 'assistant':
            prompt.append(m)
        else:
            ground_truth = m['content']
    inputs = tokenizer.apply_chat_template(prompt, add_generation_prompt=True, return_dict=True, return_tensors="pt")
    inputs = {k: v for k, v in inputs.items()}

    if ground_truth != '':
        tok = tokenizer(ground_truth, return_tensors="pt")
        out = final_model.generate(**inputs, max_new_tokens=len(tok['input_ids'][0]), do_sample=True, num_return_sequences=10)
        generations = []
        for j in range(10):
            gen = tokenizer.decode(out[j][len(inputs["input_ids"][0]):])
            generations.append(gen)
    
        t = (test_dataset[i], generations)
        post_ft.append(t)

In [ ]:
with open('experiments/EXP' + EXP + '/results_postft.pkl', 'wb') as file:
    pickle.dump(post_ft, file)

### Results

In [ ]:
with open('experiments/EXP' + EXP + '/results_postft.pkl', 'rb') as file:
    post_ft = pickle.load(file)

In [ ]:
clean = []
for d, g in post_ft:
    gen = []
    for pred in g:
        gen.append(pred.strip())
    clean.append((d, gen))

In [ ]:
mrr_list = []
for d, g in clean:
    c = d[2]['content']
    try:
        mrr = g.index(c)
    except ValueError:
        mrr = 9
    mrr_list.append(1/mrr)

mrr_mean = sum(mrr_list) / len(mrr_list)
mrr_mean

In [ ]:
hit1_list = []
for d, g in clean:
    c = d[2]['content']
    if c == g[0]:
        hit1_list.append(1)
    else:
        hit1_list.append(0)

hit1_mean = sum(hit1_list) / len(hit1_list)

hit3_list = []
for d, g in clean:
    c = d[2]['content']
    if c in g[:3]:
        hit3_list.append(1)
    else:
        hit3_list.append(0)

hit3_mean = sum(hit3_list) / len(hit3_list)

hit5_list = []
for d, g in clean:
    c = d[2]['content']
    if c in g[:5]:
        hit5_list.append(1)
    else:
        hit5_list.append(0)

hit5_mean = sum(hit5_list) / len(hit5_list)

hit7_list = []
for d, g in clean:
    c = d[2]['content']
    if c in g[:7]:
        hit7_list.append(1)
    else:
        hit7_list.append(0)

hit7_mean = sum(hit7_list) / len(hit7_list)

hit10_list = []
for d, g in clean:
    c = d[2]['content']
    if c in g:
        hit10_list.append(1)
    else:
        hit10_list.append(0)

hit10_mean = sum(hit10_list) / len(hit10_list)

print("Hit@1 = " + str(hit1_mean))
print("Hit@3 = " + str(hit3_mean))
print("Hit@5 = " + str(hit5_mean))
print("Hit@7 = " + str(hit7_mean))
print("Hit@10 = " + str(hit10_mean))